In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

# Configuration

In [2]:
dataset = "posterlayout"
task = "content"
input_format = "seq"
output_format = "html"
add_unk_token = False
add_index_token = True
add_sep_token = True
candidate_size = -1  # -1 represents the complete training set
num_prompt = 10

# Process raw data

In [ ]:
import os

from tqdm import tqdm

from preprocess import create_processor
from utils import RAW_DATA_PATH, read_pt, write_pt
from pandas import read_csv

metadata = read_csv(os.path.join(RAW_DATA_PATH(dataset), "train_csv_9973.csv"))
processor = create_processor(dataset, task, metadata=metadata)
base_dir = os.path.dirname(os.getcwd())


def get_processed_data(split):
    filename = os.path.join(
        base_dir, "dataset", dataset, "processed", task, f"{split}.pt"
    )
    if os.path.exists(filename):
        processed_data = read_pt(filename)
    else:
        processed_data = []
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        raw_path = os.path.join(RAW_DATA_PATH(dataset), split, "saliencymaps_pfpn")
        raw_data = os.listdir(raw_path)
        raw_data = sorted(raw_data, key=lambda x: int(x.split("_")[0]))
        for rd in tqdm(raw_data, desc=f"{split} data processing..."):
            idx = int(rd.split("_")[0])
            data = processor(os.path.join(raw_path, rd), idx, split)
            if data:
                processed_data.append(data)
        write_pt(filename, processed_data)
    return processed_data


processed_train_data = get_processed_data("train")
processed_test_data = get_processed_data("test")

# Dynamic exemplar selection

In [4]:
from selection import create_selector


selector = create_selector(
    task=task,
    train_data=processed_train_data,
    candidate_size=candidate_size,
    num_prompt=num_prompt,
)

test_idx = 0
exemplars = selector(processed_test_data[test_idx])

# Input-output serialization

In [5]:
from serialization import create_serializer, build_prompt


serializer = create_serializer(
    dataset=dataset,
    task=task,
    input_format=input_format,
    output_format=output_format,
    add_index_token=add_index_token,
    add_sep_token=add_sep_token,
    add_unk_token=add_unk_token
)
prompt = build_prompt(serializer, exemplars, processed_test_data[test_idx], dataset)

# Prompting Gemma

In [ ]:
%pip install unsloth
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    %pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [ ]:
# using gemma 2b quantized model
from unsloth import FastLanguageModel
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt
], return_tensors = "pt").to("cuda")

response = model.generate(**inputs, max_new_tokens = 800, use_cache = True)

# Parsing

In [ ]:
from parsing import Parser


parser = Parser(dataset=dataset, output_format=output_format)
parsed_response = parser(response)

# Layout ranking

In [305]:
from ranker import Ranker


val_path = os.path.join(RAW_DATA_PATH(dataset), "val.pt")
ranker = Ranker(val_path=val_path)
ranked_response = ranker(parsed_response)

# Visualization

In [ ]:
from visualization import Visualizer, create_image_grid


visualizer = Visualizer(dataset)
images = visualizer(ranked_response)
create_image_grid(images)